In [1]:
import glob
import time
import gc

#nativos
import os
import unicodedata
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

#terceros
import pandas as pd
import numpy as np
import pymssql
import missingno as msnolistadconoce
from collections import Counter

In [5]:
#conneciom MARKET
connection_market = pymssql.connect(
    host='10.11.12.90\BDT',
    user='ibetlmarket',
    port=1433,
    password='m@rk3t2o15',
    database='MARKET'
)

import time
time.sleep(1)
gc.collect()

44

#                                                            REPORTE DE SEGUIMIENTO GESTION ALERTAS CARTERA

In [43]:
sql_ = """
	SELECT PERIODO, SEMAFORO, ESTADO, SEGMENTO_BE_AJUST, RESULTADO FROM ODS.HD_ALERTAS_CARTERA_SEGUIMIENTO_GESTION_CARTERA
"""
data = pd.read_sql(
   sql_ , connection_market
)
data['BANCA'] = 'BE'
data['RESULTADO'] = data['RESULTADO'].apply(lambda _: str(_).upper())
data['SEMAFORO'] = data['SEMAFORO'].apply(lambda _: str(_).upper())
data['ESTADO'] = data['ESTADO'].apply(lambda _: str(_).upper())
data.head(2)

,PERIODO,SEMAFORO,ESTADO,SEGMENTO_BE_AJUST,RESULTADO,BANCA
0,202102,NEGRO,DEPURADO CLIENTE NEGRO,MEDIANA EMPRESA,NONE,BE
1,202102,NEGRO,DEPURADO CLIENTE NEGRO,GRAN EMPRESA,NONE,BE


### >> ALERTADOS (NO NEGROS)

In [17]:
subset = data[~data['SEMAFORO'].isin(['NEGRO'])]

pd.crosstab(subset['BANCA'], subset['PERIODO'])

PERIODO,202102,202103,202104
BANCA,,,
BE,502,382,356


### >> ALERTADOS POR ZONAL (NO NEGROS)

In [18]:
pd.crosstab(subset['SEGMENTO_BE_AJUST'], subset['PERIODO'])

PERIODO,202102,202103,202104
SEGMENTO_BE_AJUST,,,
GRAN EMPRESA,118,107,88
MEDIANA EMPRESA,210,154,128
PROVINCIA-S1,54,32,55
PROVINCIA-S2,120,89,85


### >> ALERTADOS POR GESTIONAR (POSTERIOR A LA DEPURACIÓN AUTOMÁTICA)


In [53]:
mismas_alertas = ['DEPURADO MISMAS ALERTAS EJECUTADO', 'DEPURADO MISMAS ALERTAS VALIDADO']
subset = data[
    (~data['SEMAFORO'].isin(['NEGRO', 'GRIS'])) & (~data['ESTADO'].isin(mismas_alertas))
]

pd.crosstab(subset['SEGMENTO_BE_AJUST'], subset['PERIODO'])

PERIODO,202102,202103,202104
SEGMENTO_BE_AJUST,,,
GRAN EMPRESA,88,45,43
MEDIANA EMPRESA,210,90,53
PROVINCIA-S1,47,14,9
PROVINCIA-S2,110,32,35


### >> REQUIERE GESTION

In [55]:
list_depura =['DEPURADO MISMAS ALERTAS EJECUTADO', 'DEPURADO MISMAS ALERTAS VALIDADO', 'EN PROCESO DE DEPURACION', 'DEPURADO']
subset = data[
    (~data['SEMAFORO'].isin(['NEGRO', 'GRIS'])) & (~data['ESTADO'].isin(list_depura))
]

pd.crosstab(subset['SEGMENTO_BE_AJUST'], subset['PERIODO'])

PERIODO,202102,202103,202104
SEGMENTO_BE_AJUST,,,
GRAN EMPRESA,50,31,43
MEDIANA EMPRESA,210,90,53
PROVINCIA-S1,33,11,7
PROVINCIA-S2,89,23,30


### >> GESTIONADOS

In [36]:
gestionado = ['EJECUTADO', 'VALIDADO', 'GUARDADO']
subset = data[
    (~data['SEMAFORO'].isin(['NEGRO', 'GRIS'])) & (data['ESTADO'].isin(gestionado))
]

pd.crosstab(subset['SEGMENTO_BE_AJUST'], subset['PERIODO'])

PERIODO,202102,202103,202104
SEGMENTO_BE_AJUST,,,
GRAN EMPRESA,25,29,3
MEDIANA EMPRESA,95,88,8
PROVINCIA-S1,7,11,0
PROVINCIA-S2,33,22,11


### >> RESULTADOS

In [51]:
subset = data[data['RESULTADO'].isin(['SEGUIMIENTO', 'PREOCUPA '])]

pd.crosstab(subset['SEGMENTO_BE_AJUST'], subset['PERIODO'])

PERIODO,202102,202103,202104
SEGMENTO_BE_AJUST,,,
GRAN EMPRESA,2,0,0
MEDIANA EMPRESA,8,5,2
PROVINCIA-S2,3,4,4
